In [31]:
import os
import numpy as np
import pandas as pd

# from unidecode import unidecode

import pickle # saving and loading trained model
from os import path

# importing required libraries for normalizing data
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

# importing library for plotting
import matplotlib.pyplot as plt
import seaborn as sn

from sklearn import metrics
from sklearn.metrics import accuracy_score # for calculating accuracy of model
from sklearn.model_selection import train_test_split # for splitting the dataset for training and testing
from sklearn.metrics import classification_report # for generating a classification report of model

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc

import tensorflow as tf
from tensorflow import keras

import tensorflow  as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Activation
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model

In [32]:
import glob

path = r'D:/PhD exp/Datasets/CIC-IDS2017' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, encoding='cp1252', index_col=None, header=0, low_memory=False)
    li.append(df)
    print("Read Completed for ", filename)

df = pd.concat(li, axis=0, ignore_index=True)

df = df.rename(columns={' Label': 'Label'})

Read Completed for  D:/PhD exp/Datasets/CIC-IDS2017\Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv
Read Completed for  D:/PhD exp/Datasets/CIC-IDS2017\Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv
Read Completed for  D:/PhD exp/Datasets/CIC-IDS2017\Friday-WorkingHours-Morning.pcap_ISCX.csv
Read Completed for  D:/PhD exp/Datasets/CIC-IDS2017\Monday-WorkingHours.pcap_ISCX.csv
Read Completed for  D:/PhD exp/Datasets/CIC-IDS2017\Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv
Read Completed for  D:/PhD exp/Datasets/CIC-IDS2017\Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv
Read Completed for  D:/PhD exp/Datasets/CIC-IDS2017\Tuesday-WorkingHours.pcap_ISCX.csv
Read Completed for  D:/PhD exp/Datasets/CIC-IDS2017\Wednesday-workingHours.pcap_ISCX.csv


In [33]:
df.describe()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,...,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06
mean,8.071483e+03,1.478566e+07,9.361160e+00,1.039377e+01,5.493024e+02,1.616264e+04,2.075999e+02,1.871366e+01,5.820194e+01,6.891013e+01,...,5.418218e+00,-2.741688e+03,8.155132e+04,4.113412e+04,1.531825e+05,5.829582e+04,8.316037e+06,5.038439e+05,8.695752e+06,7.920031e+06
std,1.828363e+04,3.365374e+07,7.496728e+02,9.973883e+02,9.993589e+03,2.263088e+06,7.171848e+02,6.033935e+01,1.860912e+02,2.811871e+02,...,6.364257e+02,1.084989e+06,6.485999e+05,3.933815e+05,1.025825e+06,5.770923e+05,2.363008e+07,4.602984e+06,2.436689e+07,2.336342e+07
min,0.000000e+00,-1.300000e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,-5.368707e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.300000e+01,1.550000e+02,2.000000e+00,1.000000e+00,1.200000e+01,0.000000e+00,6.000000e+00,0.000000e+00,6.000000e+00,0.000000e+00,...,0.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,8.000000e+01,3.131600e+04,2.000000e+00,2.000000e+00,6.200000e+01,1.230000e+02,3.700000e+01,2.000000e+00,3.400000e+01,0.000000e+00,...,1.000000e+00,2.400000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,4.430000e+02,3.204828e+06,5.000000e+00,4.000000e+00,1.870000e+02,4.820000e+02,8.100000e+01,3.600000e+01,5.000000e+01,2.616295e+01,...,2.000000e+00,3.200000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,6.553500e+04,1.200000e+08,2.197590e+05,2.919220e+05,1.290000e+07,6.554530e+08,2.482000e+04,2.325000e+03,5.940857e+03,7.125597e+03,...,2.135570e+05,1.380000e+02,1.100000e+08,7.420000e+07,1.100000e+08,1.100000e+08,1.200000e+08,7.690000e+07,1.200000e+08,1.200000e+08


In [34]:
df.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,55054,109,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,55055,52,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,46236,34,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,54863,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [35]:
df["Label"].value_counts()

Label
BENIGN                          2273097
DoS Hulk                         231073
PortScan                         158930
DDoS                             128027
DoS GoldenEye                     10293
FTP-Patator                        7938
SSH-Patator                        5897
DoS slowloris                      5796
DoS Slowhttptest                   5499
Bot                                1966
Web Attack ï¿½ Brute Force         1507
Web Attack ï¿½ XSS                  652
Infiltration                         36
Web Attack ï¿½ Sql Injection         21
Heartbleed                           11
Name: count, dtype: int64

In [36]:
# Replace specific values in the 'Label' column
df['Label'] = df['Label'].replace({'Web Attack ï¿½ Brute Force': 'Web Attack – Brute Force',
                                   'Web Attack ï¿½ XSS': 'Web Attack – XSS', 'Web Attack ï¿½ Sql Injection':'Web Attack – Sql Injection'})

# Verify the changes
label_counts_after_rename = df['Label'].value_counts()
print(label_counts_after_rename)

Label
BENIGN                        2273097
DoS Hulk                       231073
PortScan                       158930
DDoS                           128027
DoS GoldenEye                   10293
FTP-Patator                      7938
SSH-Patator                      5897
DoS slowloris                    5796
DoS Slowhttptest                 5499
Bot                              1966
Web Attack – Brute Force         1507
Web Attack – XSS                  652
Infiltration                       36
Web Attack – Sql Injection         21
Heartbleed                         11
Name: count, dtype: int64


In [37]:
# Filter out rows with labels other than 'BENIGN' and 'DDoS'
df = df[df['Label'].isin(['BENIGN', 'DDoS'])]

# Display the filtered DataFrame
# Display the updated label counts
print(df['Label'].value_counts())

Label
BENIGN    2273097
DDoS       128027
Name: count, dtype: int64


In [38]:
print(df.shape)

(2401124, 79)


In [39]:
#drop na values and reset index
data_clean = df.dropna().reset_index()

# Checkng for DUPLICATE values
data_clean.drop_duplicates(keep='first', inplace = True)

data_clean['Label'].value_counts()

print("Read {} rows.".format(len(data_clean)))

Read 2400715 rows.


In [40]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
# label encoding
labelencoder = LabelEncoder()
data_clean['Label'] = labelencoder.fit_transform(data_clean['Label'])

data_clean['Label'].value_counts()
data = data_clean





# Drop non-numeric columns if necessary
data = data.drop(columns=[ 'index'])  # Adjust column names as needed

# Handle infinities
data.replace([np.inf, -np.inf], np.nan, inplace=True)

# Fill NaNs with column mean
data.fillna(data.mean(), inplace=True)

# Check for any remaining NaNs
print("Remaining NaNs:", data.isna().sum().sum())

data.info()

Remaining NaNs: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400715 entries, 0 to 2400714
Data columns (total 79 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0    Destination Port             int64  
 1    Flow Duration                int64  
 2    Total Fwd Packets            int64  
 3    Total Backward Packets       int64  
 4   Total Length of Fwd Packets   int64  
 5    Total Length of Bwd Packets  int64  
 6    Fwd Packet Length Max        int64  
 7    Fwd Packet Length Min        int64  
 8    Fwd Packet Length Mean       float64
 9    Fwd Packet Length Std        float64
 10  Bwd Packet Length Max         int64  
 11   Bwd Packet Length Min        int64  
 12   Bwd Packet Length Mean       float64
 13   Bwd Packet Length Std        float64
 14  Flow Bytes/s                  float64
 15   Flow Packets/s               float64
 16   Flow IAT Mean                float64
 17   Flow IAT Std                 float64
 18   Flo

In [41]:
data.head()


,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
1,55054,109,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
2,55055,52,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
3,46236,34,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
4,54863,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0


In [42]:
from sklearn.ensemble import ExtraTreesClassifier
print("ETC")
dt = ExtraTreesClassifier(n_estimators=300, random_state=5, max_depth=30)

yy =  data['Label'].values
XX =data.drop(columns=['Label'])


# Training the model
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel


dt.fit(XX,yy)
# Get the feature importances
feature_importances = dt.feature_importances_

# Create a DataFrame with feature names and importances
df_feature_importances = pd.DataFrame({'Feature': XX.columns, 'Importance': feature_importances})

# Sort the DataFrame by importance in descending order
df_feature_importances = df_feature_importances.sort_values(by='Importance', ascending=False)

# Extract the top 30 important features
selected_features = df_feature_importances.head(30)['Feature'].tolist()

ETC


In [43]:
selected_features.append('Label')
selected_features

[' Bwd Packet Length Std',
 ' Packet Length Std',
 'Bwd Packet Length Max',
 ' Avg Bwd Segment Size',
 ' Bwd Packet Length Mean',
 ' Average Packet Size',
 ' Packet Length Variance',
 ' ACK Flag Count',
 ' Fwd IAT Std',
 ' Packet Length Mean',
 ' Max Packet Length',
 ' Fwd IAT Max',
 ' Destination Port',
 'Fwd IAT Total',
 ' PSH Flag Count',
 ' Idle Min',
 ' Flow IAT Max',
 ' min_seg_size_forward',
 ' Flow IAT Std',
 'Init_Win_bytes_forward',
 'Idle Mean',
 ' Idle Max',
 ' Flow Duration',
 'Bwd IAT Total',
 ' act_data_pkt_fwd',
 ' Min Packet Length',
 ' Fwd IAT Mean',
 ' Bwd IAT Max',
 ' URG Flag Count',
 ' Fwd Packet Length Std',
 'Label']

In [44]:
# Create a new DataFrame with the selected features
data = data[selected_features]

In [45]:
data.head()

,Bwd Packet Length Std,Packet Length Std,Bwd Packet Length Max,Avg Bwd Segment Size,Bwd Packet Length Mean,Average Packet Size,Packet Length Variance,ACK Flag Count,Fwd IAT Std,Packet Length Mean,...,Idle Max,Flow Duration,Bwd IAT Total,act_data_pkt_fwd,Min Packet Length,Fwd IAT Mean,Bwd IAT Max,URG Flag Count,Fwd Packet Length Std,Label
0,0.0,0.0,0,0.0,0.0,9.0,0.0,1,0.0,6.0,...,0,3,0,1,6,3.0,0,0,0.0,0
1,0.0,0.0,6,6.0,6.0,9.0,0.0,1,0.0,6.0,...,0,109,0,0,6,0.0,0,1,0.0,0
2,0.0,0.0,6,6.0,6.0,9.0,0.0,1,0.0,6.0,...,0,52,0,0,6,0.0,0,1,0.0,0
3,0.0,0.0,6,6.0,6.0,9.0,0.0,1,0.0,6.0,...,0,34,0,0,6,0.0,0,1,0.0,0
4,0.0,0.0,0,0.0,0.0,9.0,0.0,1,0.0,6.0,...,0,3,0,1,6,3.0,0,0,0.0,0


In [46]:
# Separate the dataset by labels
df_0 = data[data['Label'] == 0]
df_1 = data[data['Label'] == 1]

# Randomly sample from each subset
sample_0 = df_0.sample(n=330000, random_state=42)  # For label 0
sample_1 = df_1.sample(n=120000, random_state=42)  # For label 1

# Combine the samples
sampled_df = pd.concat([sample_0, sample_1])

# shuffle the resulting DataFrame
sampled_df = sampled_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [47]:
# Obtain a random sample of 450,000 rows
# sample_size = 450000
data_cicids2017 = sampled_df
# Display the shape of the random sample
print(data_cicids2017.shape)

(450000, 31)


In [48]:
print(data_cicids2017['Label'].value_counts())

Label
0    330000
1    120000
Name: count, dtype: int64


In [30]:
#save cleaned data sample to csv
data_cicids2017.to_csv('cleansample_cicids2017.csv', index=False)